In [1]:
import pandas as pd
from IPython.display import display
import os

In [2]:
TEAMNAME="3Idiots"
LANGNAME_MAPS = {
    "EN": "english",
    "DE": "german",
    "HI": "hindi"
}
FILENAME_TEMPLATE="{teamname}_{lang}_{task}_run_{run_id}.tsv"

In [3]:
df = pd.read_csv("../data/raw/training_data/english_dataset.tsv", sep="\t")
for i in range(1,4):
    task = f"task_{i}"
    df[task] = df[task].str.upper().replace("NULL", "NONE")
df.head()

,text_id,text,task_1,task_2,task_3
0,hasoc_en_1,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,NOT,NONE,NONE
1,hasoc_en_2,@politico No. We should remember very clearly ...,HOF,HATE,TIN
2,hasoc_en_3,@cricketworldcup Guess who would be the winner...,NOT,NONE,NONE
3,hasoc_en_4,Corbyn is too politically intellectual for #Bo...,NOT,NONE,NONE
4,hasoc_en_5,All the best to #TeamIndia for another swimmin...,NOT,NONE,NONE


In [4]:
df_pred = pd.read_csv("../outputs/EN/bert-base-cased/task_1/train.tsv", sep="\t")
df_pred.head()

,id,label,task_1_preds,NOT_probs,HOF_probs
0,0,NOT,NOT,0.987566,0.012434
1,1,HOF,HOF,0.083484,0.916516
2,2,NOT,NOT,0.976046,0.023954
3,3,NOT,NOT,0.970524,0.029476
4,4,NOT,NOT,0.989953,0.010047


In [5]:
df.shape, df_pred.shape

((5852, 5), (5852, 5))

## Check format on train data

In [6]:
lang = "EN"
model = "bert-base-cased"
data_key = "train"
for i in range(1,4):
    task = f"task_{i}"
    df[task] = df[task].str.upper().replace("NULL", "NONE")
    df_pred = pd.read_csv(f"../outputs/{lang}/{model}/{task}/{data_key}.tsv", sep="\t")
    print(df_pred.shape)
    pred_label = f"{task}_preds"
    df.loc[df[task] != "NONE", pred_label] = df_pred[pred_label].values
    df = df.fillna("NONE")
    display(df.head())
    display(df[pred_label].value_counts().to_frame().assign(proportion=lambda x: x[pred_label]*100./x[pred_label].sum()))

(5852, 5)


,text_id,text,task_1,task_2,task_3,task_1_preds
0,hasoc_en_1,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,NOT,NONE,NONE,NOT
1,hasoc_en_2,@politico No. We should remember very clearly ...,HOF,HATE,TIN,HOF
2,hasoc_en_3,@cricketworldcup Guess who would be the winner...,NOT,NONE,NONE,NOT
3,hasoc_en_4,Corbyn is too politically intellectual for #Bo...,NOT,NONE,NONE,NOT
4,hasoc_en_5,All the best to #TeamIndia for another swimmin...,NOT,NONE,NONE,NOT


,task_1_preds,proportion
NOT,3717,63.516746
HOF,2135,36.483254


(2261, 6)


,text_id,text,task_1,task_2,task_3,task_1_preds,task_2_preds
0,hasoc_en_1,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,NOT,NONE,NONE,NOT,NONE
1,hasoc_en_2,@politico No. We should remember very clearly ...,HOF,HATE,TIN,HOF,HATE
2,hasoc_en_3,@cricketworldcup Guess who would be the winner...,NOT,NONE,NONE,NOT,NONE
3,hasoc_en_4,Corbyn is too politically intellectual for #Bo...,NOT,NONE,NONE,NOT,NONE
4,hasoc_en_5,All the best to #TeamIndia for another swimmin...,NOT,NONE,NONE,NOT,NONE


,task_2_preds,proportion
NONE,3591,61.363636
HATE,1083,18.506494
PRFN,736,12.576897
OFFN,442,7.552973


(2261, 5)


,text_id,text,task_1,task_2,task_3,task_1_preds,task_2_preds,task_3_preds
0,hasoc_en_1,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,NOT,NONE,NONE,NOT,NONE,NONE
1,hasoc_en_2,@politico No. We should remember very clearly ...,HOF,HATE,TIN,HOF,HATE,TIN
2,hasoc_en_3,@cricketworldcup Guess who would be the winner...,NOT,NONE,NONE,NOT,NONE,NONE
3,hasoc_en_4,Corbyn is too politically intellectual for #Bo...,NOT,NONE,NONE,NOT,NONE,NONE
4,hasoc_en_5,All the best to #TeamIndia for another swimmin...,NOT,NONE,NONE,NOT,NONE,NONE


,task_3_preds,proportion
NONE,3591,61.363636
TIN,2071,35.389610
UNT,190,3.246753


## Write run submissions

In [15]:
RUN_ID_CONFIGS = [
    [
        ("EN", "bert-base-cased"),
        ("DE", "bert-base-multilingual-cased"),
        ("HI", "bert-base-multilingual-uncased"),
    ],
]

def write_run_submission_files(lang, model, run_id, data_key="test", use_task_4=False, skip_tasks=None):
    """Skip tasks is {task_id1, task_id2}"""
    df = pd.read_csv(f"../data/raw/test_data/{LANGNAME_MAPS[lang]}_dataset.tsv", sep="\t")
    display(df.head())
    submission_data_dir = os.path.join("../run_submissions/", lang, model)
    os.makedirs(submission_data_dir, exist_ok=True)
    if skip_tasks is None:
        skip_tasks = set()
    for i in range(1,4):
        if lang == "DE" and i == 3:
            print(f"Skipping {lang} task_{i}")
            continue
        task = f"task_{i}"
        #df[task] = df[task].str.upper().replace("NULL", "NONE")
        pred_file_path = f"../outputs/{lang}/{model}/{task}/{data_key}.tsv"
        if use_task_4:
            pred_file_path = f"../outputs/{lang}/{model}/task_4/{data_key}.tsv"
        df_pred = pd.read_csv(pred_file_path, sep="\t")
        display(df_pred.head())
        print(df_pred.shape)
        assert df.shape[0] == df_pred.shape[0], f"Shape mismatch: {df.shape[0]} != {df_pred.shape[0]}"
        pred_label = f"{task}_preds"
        if use_task_4:
            df[pred_label] = df_pred["task_4_preds"].str.split("-", expand=True)[i-1]
        else:
            df[pred_label] = df_pred[pred_label]
        if i in {2, 3}:
            df.loc[df["task_1_preds"] == "NOT", pred_label] = "NONE"
        #df.loc[df[task] != "NONE", f"{task}_preds"] = df_pred[f"{task}_preds"]
        #df = df.fillna("NONE")
        display(df.head())
        display(df[pred_label].value_counts().to_frame().assign(proportion=lambda x: x[pred_label]*100./x[pred_label].sum()))
        filename = FILENAME_TEMPLATE.format(teamname=TEAMNAME, lang=LANGNAME_MAPS[lang], task=i, run_id=run_id)
        if i in skip_tasks:
            print(f"Skipping writing of {lang} task_{i}.")
            continue
        df[["text_id", pred_label]].to_csv(os.path.join(submission_data_dir, filename), sep="\t", index=False)
        
    

In [8]:
RUN_ID=1
for lang, model in RUN_ID_CONFIGS[RUN_ID-1]:
    write_run_submission_files(lang, model, RUN_ID, data_key="test")

,text_id,text
0,hasoc_en_902,West Bengal Doctor Crisis: Protesting doctors ...
1,hasoc_en_416,68.5 million people have been forced to leave ...
2,hasoc_en_207,"You came, you saw .... we will look after the ..."
3,hasoc_en_595,We'll get Brexit delivered by October 31st. ...
4,hasoc_en_568,Fuck you. Go back to the dark ages you cow @IB...


,id,label,task_1_preds,NOT_probs,HOF_probs
0,0,NOT,NOT,0.985055,0.014945
1,1,NOT,NOT,0.961916,0.038083
2,2,NOT,NOT,0.948868,0.051132
3,3,NOT,NOT,0.973557,0.026443
4,4,NOT,HOF,0.007926,0.992074


(1153, 5)


,text_id,text,task_1_preds
0,hasoc_en_902,West Bengal Doctor Crisis: Protesting doctors ...,NOT
1,hasoc_en_416,68.5 million people have been forced to leave ...,NOT
2,hasoc_en_207,"You came, you saw .... we will look after the ...",NOT
3,hasoc_en_595,We'll get Brexit delivered by October 31st. ...,NOT
4,hasoc_en_568,Fuck you. Go back to the dark ages you cow @IB...,HOF


,task_1_preds,proportion
NOT,745,64.61405
HOF,408,35.38595


...

,task_2_preds,proportion
NONE,693,52.579666
PRFN,286,21.699545
HATE,261,19.802731
OFFN,78,5.918058


,id,label,task_3_preds,TIN_probs,UNT_probs
0,0,TIN,TIN,0.871761,0.128239
1,1,TIN,TIN,0.880245,0.119756
2,2,TIN,TIN,0.923862,0.076138
3,3,TIN,TIN,0.523710,0.476290
4,4,TIN,UNT,0.055885,0.944115


(1318, 5)


,text_id,text,task_1_preds,task_2_preds,task_3_preds
0,hasoc_hi_5061,"वक्त, इन्सान और इंग्लैंड का मौसम आपको कभी भी ध...",NOT,NONE,NONE
1,hasoc_hi_2090,#कांग्रेस के इस #कमीने की #करतूत को देखिए देश ...,HOF,HATE,TIN
2,hasoc_hi_2960,पाकिस्तान को फेकना था फेका गया। जो हार कर भी द...,HOF,PRFN,TIN
3,hasoc_hi_864,जो शब्द तूम आज किसी और औरत के लिए यूज कर रहे व...,NOT,NONE,NONE
4,hasoc_hi_54,नेता जी हम समाजवादी सिपाही हमेशा आपके साथ है आ...,NOT,NONE,NONE


,task_3_preds,proportion
NONE,693,52.579666
TIN,488,37.025797
UNT,137,10.394537


In [9]:
RUN_ID=2
lang, model = "HI", "bert-base-multilingual-cased"
write_run_submission_files(lang, model, RUN_ID, data_key="test")

,text_id,text
0,hasoc_hi_5061,"वक्त, इन्सान और इंग्लैंड का मौसम आपको कभी भी ध..."
1,hasoc_hi_2090,#कांग्रेस के इस #कमीने की #करतूत को देखिए देश ...
2,hasoc_hi_2960,पाकिस्तान को फेकना था फेका गया। जो हार कर भी द...
3,hasoc_hi_864,जो शब्द तूम आज किसी और औरत के लिए यूज कर रहे व...
4,hasoc_hi_54,नेता जी हम समाजवादी सिपाही हमेशा आपके साथ है आ...


,id,label,task_1_preds,NOT_probs,HOF_probs
0,0,NOT,NOT,0.995544,0.004456
1,1,NOT,HOF,0.008839,0.991161
2,2,NOT,HOF,0.002090,0.997910
3,3,NOT,NOT,0.850323,0.149677
4,4,NOT,NOT,0.995668,0.004332


(1318, 5)


,text_id,text,task_1_preds
0,hasoc_hi_5061,"वक्त, इन्सान और इंग्लैंड का मौसम आपको कभी भी ध...",NOT
1,hasoc_hi_2090,#कांग्रेस के इस #कमीने की #करतूत को देखिए देश ...,HOF
2,hasoc_hi_2960,पाकिस्तान को फेकना था फेका गया। जो हार कर भी द...,HOF
3,hasoc_hi_864,जो शब्द तूम आज किसी और औरत के लिए यूज कर रहे व...,NOT
4,hasoc_hi_54,नेता जी हम समाजवादी सिपाही हमेशा आपके साथ है आ...,NOT


,task_1_preds,proportion
NOT,687,52.124431
HOF,631,47.875569


...

,task_2_preds,proportion
NONE,687,52.124431
PRFN,285,21.623672
HATE,227,17.223065
OFFN,119,9.028832


,id,label,task_3_preds,TIN_probs,UNT_probs
0,0,TIN,TIN,0.731503,0.268497
1,1,TIN,TIN,0.754488,0.245512
2,2,TIN,TIN,0.716344,0.283656
3,3,TIN,TIN,0.661797,0.338203
4,4,TIN,TIN,0.757401,0.242599


(1318, 5)


,text_id,text,task_1_preds,task_2_preds,task_3_preds
0,hasoc_hi_5061,"वक्त, इन्सान और इंग्लैंड का मौसम आपको कभी भी ध...",NOT,NONE,NONE
1,hasoc_hi_2090,#कांग्रेस के इस #कमीने की #करतूत को देखिए देश ...,HOF,HATE,TIN
2,hasoc_hi_2960,पाकिस्तान को फेकना था फेका गया। जो हार कर भी द...,HOF,PRFN,TIN
3,hasoc_hi_864,जो शब्द तूम आज किसी और औरत के लिए यूज कर रहे व...,NOT,NONE,NONE
4,hasoc_hi_54,नेता जी हम समाजवादी सिपाही हमेशा आपके साथ है आ...,NOT,NONE,NONE


,task_3_preds,proportion
NONE,687,52.124431
TIN,610,46.282246
UNT,21,1.593323


In [10]:
RUN_ID=2
lang, model = "DE", "bert-base-multilingual-cased"
write_run_submission_files(lang, model, RUN_ID, data_key="test", use_task_4=True)

,text_id,text
0,hasoc_de_5270,"Die GrÃ¼ne Intelligenz Bestie 'Eckhardt', Pran..."
1,hasoc_de_5271,"Das Deutschlandlied,\r\nSinn und Unsinn\r\nhtt..."
2,hasoc_de_5272,#Eurovision\r\nhttps://t.co/24T8fzUI44
3,hasoc_de_5273,Unsere ESC Kandidatinnen haben selbst Schuld a...
4,hasoc_de_5274,Scheinbar bekommt Sarah's Conner's Sohn Vincen...


,id,label,task_4_preds,NOT-NONE-NONE_probs,HOF-HATE-TIN_probs,HOF-HATE-UNT_probs,HOF-OFFN-TIN_probs,HOF-OFFN-UNT_probs,HOF-PRFN-TIN_probs,HOF-PRFN-UNT_probs
0,0,NOT-NONE-NONE,NOT-NONE-NONE,0.401108,0.155253,0.001786,0.364554,0.002700,0.072980,0.001619
1,1,NOT-NONE-NONE,NOT-NONE-NONE,0.988327,0.003559,0.000340,0.003171,0.000321,0.003878,0.000405
2,2,NOT-NONE-NONE,NOT-NONE-NONE,0.988064,0.003273,0.000474,0.002875,0.000438,0.004270,0.000606
3,3,NOT-NONE-NONE,NOT-NONE-NONE,0.989356,0.003224,0.000388,0.002762,0.000350,0.003456,0.000465
4,4,NOT-NONE-NONE,NOT-NONE-NONE,0.977420,0.007962,0.000304,0.008050,0.000290,0.005663,0.000311


(850, 10)


,text_id,text,task_1_preds
0,hasoc_de_5270,"Die GrÃ¼ne Intelligenz Bestie 'Eckhardt', Pran...",NOT
1,hasoc_de_5271,"Das Deutschlandlied,\r\nSinn und Unsinn\r\nhtt...",NOT
2,hasoc_de_5272,#Eurovision\r\nhttps://t.co/24T8fzUI44,NOT
3,hasoc_de_5273,Unsere ESC Kandidatinnen haben selbst Schuld a...,NOT
4,hasoc_de_5274,Scheinbar bekommt Sarah's Conner's Sohn Vincen...,NOT


,task_1_preds,proportion
NOT,796,93.647059
HOF,54,6.352941


,id,label,task_4_preds,NOT-NONE-NONE_probs,HOF-HATE-TIN_probs,HOF-HATE-UNT_probs,HOF-OFFN-TIN_probs,HOF-OFFN-UNT_probs,HOF-PRFN-TIN_probs,HOF-PRFN-UNT_probs
0,0,NOT-NONE-NONE,NOT-NONE-NONE,0.401108,0.155253,0.001786,0.364554,0.002700,0.072980,0.001619
1,1,NOT-NONE-NONE,NOT-NONE-NONE,0.988327,0.003559,0.000340,0.003171,0.000321,0.003878,0.000405
2,2,NOT-NONE-NONE,NOT-NONE-NONE,0.988064,0.003273,0.000474,0.002875,0.000438,0.004270,0.000606
3,3,NOT-NONE-NONE,NOT-NONE-NONE,0.989356,0.003224,0.000388,0.002762,0.000350,0.003456,0.000465
4,4,NOT-NONE-NONE,NOT-NONE-NONE,0.977420,0.007962,0.000304,0.008050,0.000290,0.005663,0.000311


(850, 10)


,text_id,text,task_1_preds,task_2_preds
0,hasoc_de_5270,"Die GrÃ¼ne Intelligenz Bestie 'Eckhardt', Pran...",NOT,NONE
1,hasoc_de_5271,"Das Deutschlandlied,\r\nSinn und Unsinn\r\nhtt...",NOT,NONE
2,hasoc_de_5272,#Eurovision\r\nhttps://t.co/24T8fzUI44,NOT,NONE
3,hasoc_de_5273,Unsere ESC Kandidatinnen haben selbst Schuld a...,NOT,NONE
4,hasoc_de_5274,Scheinbar bekommt Sarah's Conner's Sohn Vincen...,NOT,NONE


,task_2_preds,proportion
NONE,796,93.647059
OFFN,54,6.352941


No german task 3. Skipping.


In [18]:
RUN_ID=2
lang, model = "EN", "bert-base-uncased"
write_run_submission_files(lang, model, RUN_ID, data_key="test", use_task_4=True)

,text_id,text
0,hasoc_en_902,West Bengal Doctor Crisis: Protesting doctors ...
1,hasoc_en_416,68.5 million people have been forced to leave ...
2,hasoc_en_207,"You came, you saw .... we will look after the ..."
3,hasoc_en_595,We'll get Brexit delivered by October 31st. ...
4,hasoc_en_568,Fuck you. Go back to the dark ages you cow @IB...


,id,label,task_4_preds,NOT-NONE-NONE_probs,HOF-HATE-TIN_probs,HOF-HATE-UNT_probs,HOF-OFFN-TIN_probs,HOF-OFFN-UNT_probs,HOF-PRFN-TIN_probs,HOF-PRFN-UNT_probs
0,0,NOT-NONE-NONE,NOT-NONE-NONE,0.925489,0.048428,0.018782,0.001577,0.003120,0.001234,0.001371
1,1,NOT-NONE-NONE,NOT-NONE-NONE,0.864343,0.102280,0.019380,0.004764,0.004051,0.003032,0.002150
2,2,NOT-NONE-NONE,NOT-NONE-NONE,0.701380,0.175041,0.058068,0.015762,0.020464,0.017056,0.012228
3,3,NOT-NONE-NONE,NOT-NONE-NONE,0.924823,0.054397,0.010791,0.002577,0.002108,0.003815,0.001490
4,4,NOT-NONE-NONE,HOF-PRFN-TIN,0.005461,0.006577,0.007182,0.017835,0.008955,0.867589,0.086401


(1153, 10)


,text_id,text,task_1_preds
0,hasoc_en_902,West Bengal Doctor Crisis: Protesting doctors ...,NOT
1,hasoc_en_416,68.5 million people have been forced to leave ...,NOT
2,hasoc_en_207,"You came, you saw .... we will look after the ...",NOT
3,hasoc_en_595,We'll get Brexit delivered by October 31st. ...,NOT
4,hasoc_en_568,Fuck you. Go back to the dark ages you cow @IB...,HOF


,task_1_preds,proportion
NOT,785,68.083261
HOF,368,31.916739


...

,task_2_preds,proportion
NONE,785,68.083261
HATE,220,19.080659
PRFN,110,9.540330
OFFN,38,3.295750


,id,label,task_4_preds,NOT-NONE-NONE_probs,HOF-HATE-TIN_probs,HOF-HATE-UNT_probs,HOF-OFFN-TIN_probs,HOF-OFFN-UNT_probs,HOF-PRFN-TIN_probs,HOF-PRFN-UNT_probs
0,0,NOT-NONE-NONE,NOT-NONE-NONE,0.925489,0.048428,0.018782,0.001577,0.003120,0.001234,0.001371
1,1,NOT-NONE-NONE,NOT-NONE-NONE,0.864343,0.102280,0.019380,0.004764,0.004051,0.003032,0.002150
2,2,NOT-NONE-NONE,NOT-NONE-NONE,0.701380,0.175041,0.058068,0.015762,0.020464,0.017056,0.012228
3,3,NOT-NONE-NONE,NOT-NONE-NONE,0.924823,0.054397,0.010791,0.002577,0.002108,0.003815,0.001490
4,4,NOT-NONE-NONE,HOF-PRFN-TIN,0.005461,0.006577,0.007182,0.017835,0.008955,0.867589,0.086401


(1153, 10)


,text_id,text,task_1_preds,task_2_preds,task_3_preds
0,hasoc_en_902,West Bengal Doctor Crisis: Protesting doctors ...,NOT,NONE,NONE
1,hasoc_en_416,68.5 million people have been forced to leave ...,NOT,NONE,NONE
2,hasoc_en_207,"You came, you saw .... we will look after the ...",NOT,NONE,NONE
3,hasoc_en_595,We'll get Brexit delivered by October 31st. ...,NOT,NONE,NONE
4,hasoc_en_568,Fuck you. Go back to the dark ages you cow @IB...,HOF,PRFN,TIN


,task_3_preds,proportion
NONE,785,68.083261
TIN,365,31.656548
UNT,3,0.260191


In [11]:
RUN_ID=3
lang, model = "HI", "bert-base-multilingual-uncased"
write_run_submission_files(lang, model, RUN_ID, data_key="test", use_task_4=True)

,text_id,text
0,hasoc_hi_5061,"वक्त, इन्सान और इंग्लैंड का मौसम आपको कभी भी ध..."
1,hasoc_hi_2090,#कांग्रेस के इस #कमीने की #करतूत को देखिए देश ...
2,hasoc_hi_2960,पाकिस्तान को फेकना था फेका गया। जो हार कर भी द...
3,hasoc_hi_864,जो शब्द तूम आज किसी और औरत के लिए यूज कर रहे व...
4,hasoc_hi_54,नेता जी हम समाजवादी सिपाही हमेशा आपके साथ है आ...


,id,label,task_4_preds,NOT-NONE-NONE_probs,HOF-HATE-TIN_probs,HOF-HATE-UNT_probs,HOF-OFFN-TIN_probs,HOF-OFFN-UNT_probs,HOF-PRFN-TIN_probs,HOF-PRFN-UNT_probs
0,0,NOT-NONE-NONE,NOT-NONE-NONE,0.974389,0.009490,0.005164,0.002407,0.005468,0.001652,0.001428
1,1,NOT-NONE-NONE,HOF-HATE-TIN,0.114733,0.356426,0.097945,0.201068,0.156922,0.045574,0.027332
2,2,NOT-NONE-NONE,HOF-PRFN-TIN,0.006509,0.021342,0.005826,0.040227,0.026821,0.490624,0.408651
3,3,NOT-NONE-NONE,HOF-OFFN-TIN,0.074323,0.210795,0.075234,0.268983,0.188031,0.108094,0.074541
4,4,NOT-NONE-NONE,NOT-NONE-NONE,0.974262,0.009827,0.005671,0.002384,0.005183,0.001417,0.001257


(1318, 10)


,text_id,text,task_1_preds
0,hasoc_hi_5061,"वक्त, इन्सान और इंग्लैंड का मौसम आपको कभी भी ध...",NOT
1,hasoc_hi_2090,#कांग्रेस के इस #कमीने की #करतूत को देखिए देश ...,HOF
2,hasoc_hi_2960,पाकिस्तान को फेकना था फेका गया। जो हार कर भी द...,HOF
3,hasoc_hi_864,जो शब्द तूम आज किसी और औरत के लिए यूज कर रहे व...,HOF
4,hasoc_hi_54,नेता जी हम समाजवादी सिपाही हमेशा आपके साथ है आ...,NOT


,task_1_preds,proportion
NOT,757,57.435508
HOF,561,42.564492


...

,task_2_preds,proportion
NONE,757,57.435508
PRFN,262,19.878604
HATE,159,12.063733
OFFN,140,10.622155


,id,label,task_4_preds,NOT-NONE-NONE_probs,HOF-HATE-TIN_probs,HOF-HATE-UNT_probs,HOF-OFFN-TIN_probs,HOF-OFFN-UNT_probs,HOF-PRFN-TIN_probs,HOF-PRFN-UNT_probs
0,0,NOT-NONE-NONE,NOT-NONE-NONE,0.974389,0.009490,0.005164,0.002407,0.005468,0.001652,0.001428
1,1,NOT-NONE-NONE,HOF-HATE-TIN,0.114733,0.356426,0.097945,0.201068,0.156922,0.045574,0.027332
2,2,NOT-NONE-NONE,HOF-PRFN-TIN,0.006509,0.021342,0.005826,0.040227,0.026821,0.490624,0.408651
3,3,NOT-NONE-NONE,HOF-OFFN-TIN,0.074323,0.210795,0.075234,0.268983,0.188031,0.108094,0.074541
4,4,NOT-NONE-NONE,NOT-NONE-NONE,0.974262,0.009827,0.005671,0.002384,0.005183,0.001417,0.001257


(1318, 10)


,text_id,text,task_1_preds,task_2_preds,task_3_preds
0,hasoc_hi_5061,"वक्त, इन्सान और इंग्लैंड का मौसम आपको कभी भी ध...",NOT,NONE,NONE
1,hasoc_hi_2090,#कांग्रेस के इस #कमीने की #करतूत को देखिए देश ...,HOF,HATE,TIN
2,hasoc_hi_2960,पाकिस्तान को फेकना था फेका गया। जो हार कर भी द...,HOF,PRFN,TIN
3,hasoc_hi_864,जो शब्द तूम आज किसी और औरत के लिए यूज कर रहे व...,HOF,OFFN,TIN
4,hasoc_hi_54,नेता जी हम समाजवादी सिपाही हमेशा आपके साथ है आ...,NOT,NONE,NONE


,task_3_preds,proportion
NONE,757,57.435508
TIN,548,41.578149
UNT,13,0.986343


In [16]:
RUN_ID=3
lang, model = "DE", "bert-base-german-cased"
write_run_submission_files(lang, model, RUN_ID, data_key="test", use_task_4=True, skip_tasks={2, 3})

,text_id,text
0,hasoc_de_5270,"Die GrÃ¼ne Intelligenz Bestie 'Eckhardt', Pran..."
1,hasoc_de_5271,"Das Deutschlandlied,\r\nSinn und Unsinn\r\nhtt..."
2,hasoc_de_5272,#Eurovision\r\nhttps://t.co/24T8fzUI44
3,hasoc_de_5273,Unsere ESC Kandidatinnen haben selbst Schuld a...
4,hasoc_de_5274,Scheinbar bekommt Sarah's Conner's Sohn Vincen...


,id,label,task_4_preds,NOT-NONE-NONE_probs,HOF-HATE-TIN_probs,HOF-HATE-UNT_probs,HOF-OFFN-TIN_probs,HOF-OFFN-UNT_probs,HOF-PRFN-TIN_probs,HOF-PRFN-UNT_probs
0,0,NOT-NONE-NONE,NOT-NONE-NONE,0.977112,0.014205,0.000352,0.006218,0.000360,0.001418,0.000335
1,1,NOT-NONE-NONE,NOT-NONE-NONE,0.997487,0.000544,0.000182,0.000441,0.000224,0.000891,0.000230
2,2,NOT-NONE-NONE,NOT-NONE-NONE,0.997163,0.000571,0.000236,0.000417,0.000270,0.001038,0.000305
3,3,NOT-NONE-NONE,NOT-NONE-NONE,0.995819,0.000764,0.000164,0.001000,0.000186,0.001889,0.000179
4,4,NOT-NONE-NONE,NOT-NONE-NONE,0.996108,0.000699,0.000251,0.000758,0.000267,0.001631,0.000286


(850, 10)


,text_id,text,task_1_preds
0,hasoc_de_5270,"Die GrÃ¼ne Intelligenz Bestie 'Eckhardt', Pran...",NOT
1,hasoc_de_5271,"Das Deutschlandlied,\r\nSinn und Unsinn\r\nhtt...",NOT
2,hasoc_de_5272,#Eurovision\r\nhttps://t.co/24T8fzUI44,NOT
3,hasoc_de_5273,Unsere ESC Kandidatinnen haben selbst Schuld a...,NOT
4,hasoc_de_5274,Scheinbar bekommt Sarah's Conner's Sohn Vincen...,NOT


,task_1_preds,proportion
NOT,806,94.823529
HOF,44,5.176471


,id,label,task_4_preds,NOT-NONE-NONE_probs,HOF-HATE-TIN_probs,HOF-HATE-UNT_probs,HOF-OFFN-TIN_probs,HOF-OFFN-UNT_probs,HOF-PRFN-TIN_probs,HOF-PRFN-UNT_probs
0,0,NOT-NONE-NONE,NOT-NONE-NONE,0.977112,0.014205,0.000352,0.006218,0.000360,0.001418,0.000335
1,1,NOT-NONE-NONE,NOT-NONE-NONE,0.997487,0.000544,0.000182,0.000441,0.000224,0.000891,0.000230
2,2,NOT-NONE-NONE,NOT-NONE-NONE,0.997163,0.000571,0.000236,0.000417,0.000270,0.001038,0.000305
3,3,NOT-NONE-NONE,NOT-NONE-NONE,0.995819,0.000764,0.000164,0.001000,0.000186,0.001889,0.000179
4,4,NOT-NONE-NONE,NOT-NONE-NONE,0.996108,0.000699,0.000251,0.000758,0.000267,0.001631,0.000286


(850, 10)


,text_id,text,task_1_preds,task_2_preds
0,hasoc_de_5270,"Die GrÃ¼ne Intelligenz Bestie 'Eckhardt', Pran...",NOT,NONE
1,hasoc_de_5271,"Das Deutschlandlied,\r\nSinn und Unsinn\r\nhtt...",NOT,NONE
2,hasoc_de_5272,#Eurovision\r\nhttps://t.co/24T8fzUI44,NOT,NONE
3,hasoc_de_5273,Unsere ESC Kandidatinnen haben selbst Schuld a...,NOT,NONE
4,hasoc_de_5274,Scheinbar bekommt Sarah's Conner's Sohn Vincen...,NOT,NONE


,task_2_preds,proportion
NONE,806,94.823529
OFFN,21,2.470588
HATE,18,2.117647
PRFN,5,0.588235


Skipping writing of DE task_2.
Skipping DE task_3


In [17]:
RUN_ID=3
lang, model = "DE", "bert-base-german-cased"
write_run_submission_files(lang, model, RUN_ID, data_key="test", skip_tasks={1, 3})

,text_id,text
0,hasoc_de_5270,"Die GrÃ¼ne Intelligenz Bestie 'Eckhardt', Pran..."
1,hasoc_de_5271,"Das Deutschlandlied,\r\nSinn und Unsinn\r\nhtt..."
2,hasoc_de_5272,#Eurovision\r\nhttps://t.co/24T8fzUI44
3,hasoc_de_5273,Unsere ESC Kandidatinnen haben selbst Schuld a...
4,hasoc_de_5274,Scheinbar bekommt Sarah's Conner's Sohn Vincen...


,id,label,task_1_preds,NOT_probs,HOF_probs
0,0,NOT,NOT,0.996675,0.003325
1,1,NOT,NOT,0.999717,0.000283
2,2,NOT,NOT,0.999781,0.000219
3,3,NOT,NOT,0.997951,0.002049
4,4,NOT,NOT,0.999693,0.000307


(850, 5)


,text_id,text,task_1_preds
0,hasoc_de_5270,"Die GrÃ¼ne Intelligenz Bestie 'Eckhardt', Pran...",NOT
1,hasoc_de_5271,"Das Deutschlandlied,\r\nSinn und Unsinn\r\nhtt...",NOT
2,hasoc_de_5272,#Eurovision\r\nhttps://t.co/24T8fzUI44,NOT
3,hasoc_de_5273,Unsere ESC Kandidatinnen haben selbst Schuld a...,NOT
4,hasoc_de_5274,Scheinbar bekommt Sarah's Conner's Sohn Vincen...,NOT


,task_1_preds,proportion
NOT,806,94.823529
HOF,44,5.176471


...

,id,label,task_2_preds,HATE_probs,OFFN_probs,PRFN_probs
0,0,HATE,OFFN,0.226050,0.710878,0.063072
1,1,HATE,OFFN,0.242966,0.400706,0.356328
2,2,HATE,PRFN,0.204481,0.326761,0.468757
3,3,HATE,OFFN,0.174593,0.630145,0.195262
4,4,HATE,OFFN,0.133691,0.541420,0.324889


(850, 6)


,text_id,text,task_1_preds,task_2_preds
0,hasoc_de_5270,"Die GrÃ¼ne Intelligenz Bestie 'Eckhardt', Pran...",NOT,NONE
1,hasoc_de_5271,"Das Deutschlandlied,\r\nSinn und Unsinn\r\nhtt...",NOT,NONE
2,hasoc_de_5272,#Eurovision\r\nhttps://t.co/24T8fzUI44,NOT,NONE
3,hasoc_de_5273,Unsere ESC Kandidatinnen haben selbst Schuld a...,NOT,NONE
4,hasoc_de_5274,Scheinbar bekommt Sarah's Conner's Sohn Vincen...,NOT,NONE


,task_2_preds,proportion
NONE,806,94.823529
OFFN,39,4.588235
HATE,5,0.588235


Skipping DE task_3


In [19]:
RUN_ID=3
lang, model = "EN", "bert-base-uncased"
write_run_submission_files(lang, model, RUN_ID, data_key="test", use_task_4=False)

,text_id,text
0,hasoc_en_902,West Bengal Doctor Crisis: Protesting doctors ...
1,hasoc_en_416,68.5 million people have been forced to leave ...
2,hasoc_en_207,"You came, you saw .... we will look after the ..."
3,hasoc_en_595,We'll get Brexit delivered by October 31st. ...
4,hasoc_en_568,Fuck you. Go back to the dark ages you cow @IB...


,id,label,task_1_preds,NOT_probs,HOF_probs
0,0,NOT,NOT,0.981373,0.018627
1,1,NOT,NOT,0.922009,0.077991
2,2,NOT,NOT,0.941028,0.058972
3,3,NOT,NOT,0.989565,0.010435
4,4,NOT,HOF,0.005149,0.994851


(1153, 5)


,text_id,text,task_1_preds
0,hasoc_en_902,West Bengal Doctor Crisis: Protesting doctors ...,NOT
1,hasoc_en_416,68.5 million people have been forced to leave ...,NOT
2,hasoc_en_207,"You came, you saw .... we will look after the ...",NOT
3,hasoc_en_595,We'll get Brexit delivered by October 31st. ...,NOT
4,hasoc_en_568,Fuck you. Go back to the dark ages you cow @IB...,HOF


,task_1_preds,proportion
NOT,734,63.660017
HOF,419,36.339983


...

,task_2_preds,proportion
NONE,734,63.660017
HATE,245,21.248916
PRFN,118,10.234172
OFFN,56,4.856895


,id,label,task_3_preds,TIN_probs,UNT_probs
0,0,TIN,TIN,0.962873,0.037127
1,1,TIN,UNT,0.064256,0.935744
2,2,TIN,TIN,0.620635,0.379365
3,3,TIN,TIN,0.857289,0.142711
4,4,TIN,TIN,0.962777,0.037223


(1153, 5)


,text_id,text,task_1_preds,task_2_preds,task_3_preds
0,hasoc_en_902,West Bengal Doctor Crisis: Protesting doctors ...,NOT,NONE,NONE
1,hasoc_en_416,68.5 million people have been forced to leave ...,NOT,NONE,NONE
2,hasoc_en_207,"You came, you saw .... we will look after the ...",NOT,NONE,NONE
3,hasoc_en_595,We'll get Brexit delivered by October 31st. ...,NOT,NONE,NONE
4,hasoc_en_568,Fuck you. Go back to the dark ages you cow @IB...,HOF,PRFN,TIN


,task_3_preds,proportion
NONE,734,63.660017
TIN,348,30.182134
UNT,71,6.157849
